In [1]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.models import Sequential,Model
from keras.layers import LSTM,Input
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector

# generate a sequence of random integers
def generate_sequence(length, n_unique):
	return [randint(0, n_unique-1) for _ in range(length)]

# one hot encode sequence
def one_hot_encode(sequence, n_unique):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[value] = 1
		encoding.append(vector)
	return array(encoding)

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]

# prepare data for the LSTM
def get_pair(n_in, n_out, cardinality):
	# generate random sequence
	sequence_in = generate_sequence(n_in, cardinality)
	sequence_out = sequence_in[:n_out] + [0 for _ in range(n_in-n_out)]
	# one hot encode
	X = one_hot_encode(sequence_in, cardinality)
	y = one_hot_encode(sequence_out, cardinality)
	# reshape as 3D
	X = X.reshape((1, X.shape[0], X.shape[1]))
	y = y.reshape((1, y.shape[0], y.shape[1]))
	return X,y

Using TensorFlow backend.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# configure problem
n_features = 50
n_timesteps_in = 5
n_timesteps_out = 2
X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
X.shape

(1, 5, 50)

In [3]:
input_data = Input(shape=[n_timesteps_in, n_features])
l1 = LSTM(150,return_sequences=True)(input_data)
# r = RepeatVector(n_timesteps_in)(l1)

l2 = LSTM(150, return_sequences=True)(l1)
d1 = Dense(n_features, activation='softmax')(l2)


model = Model(inputs=input_data,outputs=d1)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
print(model.summary())
# print(model.predict(X).shape)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5, 50)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 150)            120600    
_________________________________________________________________
lstm_2 (LSTM)                (None, 5, 150)            180600    
_________________________________________________________________
dense_1 (Dense)              (None, 5, 50)             7550      
Total params: 308,750
Trainable params: 308,750
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
# train LSTM
for epoch in range(5000):
	# generate new random sequence
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	# fit model for one epoch on this sequence
	model.fit(X, y, epochs=1, verbose=2)
# evaluate LSTM
total, correct = 100, 0
for _ in range(total):
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	yhat = model.predict(X, verbose=0)
	if array_equal(one_hot_decode(y[0]), one_hot_decode(yhat[0])):
		correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))
# spot check some examples
for _ in range(10):
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	yhat = model.predict(X, verbose=0)
	print('Expected:', one_hot_decode(y[0]), 'Predicted', one_hot_decode(yhat[0]))

Epoch 1/1
 - 2s - loss: 3.9116 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.9071 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.9052 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.8892 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8763 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8670 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8473 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 3.8277 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 3.8279 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8161 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8111 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7718 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7790 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7443 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7099 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.6568 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.6512 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.5713 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.5013 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.4400 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.3623 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.2496 - acc: 0.600

Epoch 1/1
 - 0s - loss: 1.6235 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6490 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5990 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6000 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5743 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5493 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6194 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6052 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5820 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4862 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5889 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5859 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6252 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6426 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.2842 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.4868 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5361 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6260 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5530 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5674 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6062 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5752 - acc: 0.6000
Epoch 1/1


 - 0s - loss: 1.6375 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5445 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.6340 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5415 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5877 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5475 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6312 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5571 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5630 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5921 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5878 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5607 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6239 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5494 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5218 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5849 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5323 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5525 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5679 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6658 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5247 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5567 - acc: 0.6000
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 1.4604 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.4915 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5751 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5339 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5395 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5030 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5894 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5399 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6205 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5672 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5483 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.2878 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.4644 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6073 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5437 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5297 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5985 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4773 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5288 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5936 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5026 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4494 - acc: 0.8000
Epoch 1/1


 - 0s - loss: 1.4040 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5593 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.2528 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.3960 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.4924 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4873 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5805 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5530 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.3079 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.4487 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5328 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.4917 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4056 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.2434 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.4261 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8166 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.4261 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5676 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4764 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5235 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.3169 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.5316 - acc: 0.8000
Epoch 1/1
 - 0s - lo

...

Epoch 1/1
 - 0s - loss: 0.0013 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0014 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0012 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0026 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0014 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0012 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0011 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0013 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0010 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 8.7002e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0013 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0011 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 8.7596e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0011 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0012 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0014 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0011 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0013 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 8.4558e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0012 - acc: 1.000

Epoch 1/1
 - 0s - loss: 0.0081 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0057 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0031 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0040 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0148 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0068 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0102 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0141 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0646 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0167 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0053 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0040 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0040 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.4133 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0047 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0044 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0021 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0088 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0017 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0010 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0019 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0040 - acc: 1.0000
Epoch 1/1


 - 0s - loss: 0.0016 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0021 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0017 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 9.6750e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 7.4798e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0016 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0029 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0014 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0018 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0018 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0013 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0047 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 8.4527e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 5.7957e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0010 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0016 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0040 - acc: 1.0000
Epoc

 - 0s - loss: 0.0010 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0018 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 7.1842e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0070 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0016 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0011 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0011 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 2.1365e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 3.6609e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0010 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0025 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0014 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 8.6208e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 5.2218e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 5.5292e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 8.7865e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0012 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0014 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 9.9049e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0013 - 

Epoch 1/1
 - 0s - loss: 4.3801e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0047 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 7.1468e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 8.2342e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0017 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0011 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 8.5968e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0010 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 7.2117e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 5.0594e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0019 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 8.1661e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0012 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 9.5273e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 6.1814e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0012 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0020 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 8.0322e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0012 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0014 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 8.8548e-04 - acc: 1.0000
Epoch 1/1
 